## Problem set 2: Naive Bayes classification

Answer each question. You may discuss answers with other students, but write your own solution. Some questions have more than one part. Most questions involve coding, some require you to comment.

Can you tell whether a review of a restaurant is positive or negative? What words are most indicative? We'll examine data from Yelp, provided as part of the Yelp Academic Challenge.

To do this, we will create a program (a classifier) that uses actual examples of one-star and five-star reviews to classify a new review as positive or negative.  This classifier will estimate the probability that a review is negative or positive based on the words in that review and assign the most likely label (negative or positive).

Some terminology: the word "word" can mean two different things, so we will be more specific. The phrase "red fish blue fish" contains a sequence of four word *tokens*, but only three distinct word *types* (red, fish, blue). 

### Q1: guesses before looking at data

Before we start coding, list five word types that you think stongly indicate a positive review and five that indicate a negative review:

Positive words here: 
Negative words here: 

Now we will create a program that uses actual examples of one-star and five-star reviews to classify text as positive and negative.

We start by partioning our collection of pre-labeled restaurant reviews into two sets, positive reviews and negative reviews. (We have done this for you.) We will then count the relative frequency of each distinct word type in the two sub-collections which will allow us to estimate the probability that a given word appears in a review given its polarity (positive or negative). These individual word probabilites will then be multiplied to etimate the probability of multiple word tokens appearing in a document given its polarity. Finally, we can then appply Bayes theorem to switch our conditionals: i.e. to estimate the probability a review is positive or negative based on the words in it.

Here's the math. The first line is an application of Bayes' rule. The second line expands the marginal probability of the words into the sum over positive and negative polarities. The third line adds the "naive" assumption that words are independent, as long as we know the polarity. (Note that I flipped the order of the prior probability and the word probability in the third line, to make it clear where the product ends.)

\begin{align}
P(positive|w_1, w_2, w_3, ..., w_N) & = \frac{P(w_1, w_2, w_3, ..., w_N | positive)P(positive)}{P(w_1, w_2, w_3, ..., w_N)} \\
&= \frac{P(w_1, w_2, w_3, ..., w_N | positive)P(positive)}{P(w_1, w_2, w_3, ..., w_N | positive)P(positive) + P(w_1, w_2, w_3, ..., w_N | negative)P(negative)} \\
&\approx \frac{P(positive)\prod_i P(w_i|positive)}{P(positive)\prod_i P(w_i|positive) + P(negative)\prod_i P(w_i|negative) }
\end{align}

### Q2: create a pattern

The first step is to count the words in each of the two *training* sub-collections. They are in two files, `positive.txt` and `negative.txt`. Each line in these files is one review. Before we start, we need to decide how we are going to break reviews into words.

Create a regular expression that matches sequences of one or more letter characters. Then write a short sentence and use the `findall` function to list all of the substrings of that sentence that match your pattern. Include at least one common word in your sentence that will not be handled well by this pattern.

In [12]:
import re
my_reg_exp = re.compile("\w+")
my_reg_exp.findall("Can't do this")

['Can', 't', 'do', 'this']

### Q3: Specify variables to store data

To apply Bayes' rule to estimate the probability of polarity given a review, we need the probability of words given polarity and the probability of positive or negative polarity.

First create variables that will store this information.
For the positive reviews create a `Counter` that will store the counts of words in positive reviews. Also create a variable that will count the number of positive reviews. Create similar variables for the negative reviews.

In [13]:
from collections import Counter
pos_word_counts = Counter()
neg_word_counts = Counter()
num_pos_reviews = 0
num_neg_reviews = 0

### Q4: Read data from files

Now read the two files, collecting the count of each distinct word in each sub-collection, as well as the number of reviews in each sub-collection. You will need to *update* the appropriate `Counter` for each review.

Once you have processed the two files, test the counters by printing the five most common words in each counter.

In [14]:
# import re

with open("positive.txt", 'r') as pos_reader:
    my_reg_exp = re.compile("\w+")
    for line in pos_reader:
        ## update word counts in positive reviews
        pos_word_counts.update(my_reg_exp.findall(line))
        ## update our count for the number of positive reviews
        num_pos_reviews += 1

# print(pos_word_counts)
pos_word_counts_dict = dict(pos_word_counts)
# print(pos_word_counts_dict)
sort = sorted(pos_word_counts_dict, key=pos_word_counts_dict.get)
# print(sort)

for i in range((len(sort)-6), (len(sort)-1)):
    print(sort[i])
    
with open("negative.txt", 'r') as neg_reader:
    my_reg_exp = re.compile("\w+")
    try:
        for line in neg_reader:
            neg_word_counts.update(my_reg_exp.findall(line))
            num_neg_reviews += 1
    except UnicodeDecodeError:
        x=4
print( num_pos_reviews)       
neg_word_counts_dict = dict(neg_word_counts)
sort2 = sorted(neg_word_counts_dict, key=neg_word_counts_dict.get)
# print(sort2)

for i in range(len(sort2)-6, len(sort2)-1):
    print(sort2[i])
        
## for document in documents:
## pos_word_counts.update()
## pos_document_counts += 1

of
to
a
I
and
3883
was
a
to
and
I


In [5]:
# try:
#     read_a_file("file.txt")
# except UnicodeDecodeerror:
#     continue

SyntaxError: 'continue' not properly in loop (<ipython-input-5-53e2a34dd854>, line 4)

### Q5: prior probability

Calculate the probability that a randomly selected review is positive. 

In [15]:
# prob_pos = of positive reviews / (# of reviews)
prob_pos = num_pos_reviews / (num_pos_reviews + num_neg_reviews)
print(prob_pos)

0.8309437192381768


Does this value surprise you? Why or why not?

Answer here: I think it's good that there are a good amount more positive reviews than negative reviews. I feel like in general people who are dissatisfied comment with bad reviews, while the satisfied people don't comment. 

### Q6: count word totals

Create two variables, `positive_tokens` and `negative_tokens`, whose value is the total number of word tokens in that sub-collection. Print these values.

In [16]:
pos_tokens = 0
for i in sort:
    number = pos_word_counts_dict.get(i)
    pos_tokens += number
    
print(pos_tokens)

neg_tokens = 0
for i in sort2:
    number = neg_word_counts_dict.get(i)
    neg_tokens += number
    
print(neg_tokens)

415765
126358


### Q7: calculate word probability (version 1.0)

Create a function `word_prob` that takes three arguments: a word, a counter, and a sum. It should return the probability of the word estimated from that counter. Assume that the sum is accurate, you don't need to check for errors.

Use this function to print the probability of the words "delicious", "manager", "edgy", and "vile" in both sub-collections.

In [17]:
def word_prob(word, counter, sum):
    counter_dict = dict(counter)
    if counter_dict.get(word) == None:
        prob = 0
        return prob
    occur = counter_dict.get(word)
    prob = occur/sum
    
    return prob

print(word_prob("delicious", pos_word_counts, pos_tokens))
print(word_prob("delicious", neg_word_counts, neg_tokens))
print(word_prob("manager", pos_word_counts, pos_tokens))
print(word_prob("manager", neg_word_counts, neg_tokens))
print(word_prob("edgy", pos_word_counts, pos_tokens))
print(word_prob("edgy", neg_word_counts, neg_tokens))
print(word_prob("vile", pos_word_counts, pos_tokens))
print(word_prob("vile", neg_word_counts, neg_tokens))

0.0012819741921518166
5.539815445005461e-05
0.00016836434043269636
0.001044650912486744
4.810409726648467e-06
0
0
7.914022064293515e-06


### Q8: probability of a sequence (version 1.0)

Now create a function `review_prob` that takes a string containing multiple words (for example "the food was delicious"), a counter, and a sum. It should use the same regular expression method you used when reading the files earlier to break this string into tokens, and then use `word_prob` to calculate the probability of each of those word tokens. It should then return the product of those probabilities.

Print the probability of "I loved the carnitas", "but then the manager came out and told us", and "the ambience was edgy but the food was vile" according to both sub-collections. What do you notice about these six probabilities?

In [18]:
def review_prob(words, counter, sum):
    eachword = words.split(' ')
    prob = 1
    for i in eachword:
        prob *= word_prob(i, counter, sum)
        
    return prob

print(review_prob("I loved the carnitas", pos_word_counts, pos_tokens))
print(review_prob("I loved the carnitas", neg_word_counts, neg_tokens))
print(review_prob("but then the manager came out and told us", pos_word_counts, pos_tokens))
print(review_prob("but then the manager came out and told us", neg_word_counts, neg_tokens))
print(review_prob("the ambience was edgy but the food was vile", pos_word_counts, pos_tokens))
print(review_prob("the ambience was edgy but the food was vile", neg_word_counts, neg_tokens))

1.2250003720826804e-11
7.596215739715786e-13
9.500532817312996e-25
2.2807647439322206e-22
0.0
0.0


Write your observations about the six probabilities here:

The probabilities are very small, which makes sense because these are random sentences that don't have much to do with the content of reviews, so the likelihood of them popping up is very small. Additionally, it is less likely for a specific order of words to appear over singular words. 

### Q9: calculate word probability (version 2.0)

Instead of multiplying probabilities, we can also add *log* probabilities. Create a function `word_log_prob` that calculates the log probability of a word, and a function `review_log_prob` that calculates the sum of the log probabilities of the words in a string. These should take the same arguments as the previous functions.

Print the log probabilities of the same example words and sentences from the previous problems. Explain any errors that occur in comments.

In [19]:
import math

def word_log_prob(word, counter, sum):
    prob = word_prob(word, counter, sum)
    log = math.log(prob)
    return log
    
def review_log_prob(words, counter, sum):
    eachword = words.split(' ')
    prob = 0
    for i in eachword:
        prob += math.log(word_prob(i, counter, sum))
    return prob

print(word_log_prob("delicious", pos_word_counts, pos_tokens))
print(review_log_prob("I loved the carnitas", pos_word_counts, pos_tokens))

-6.659354051613109
-25.125494875196893


If a word type never occurs in our negative reviews, it will have probability zero. But we don't want to rule out the possibility that any future negative review could include that word type. We need to a way to give non-zero probability in both positive and negative polarities to a word that we have only seen in one. A simple way to do this is to add 1 to the count when we calculate word probability, as long as the word has appeared in at least one sub-collection (words we have never seen in either sub-collection will still have probability zero).

This change avoids log-zero errors, but it violates the laws of probability. Why?

Answer here: Probability is a number between 0 and 1, so adding 1 even once would make the probability of something happening at least 100%, much likely greater. 

### Q10: Collection statistics

Create a `Counter` called `both_counts` by adding the two negative and positive counters together (just use +, Counters are awesome). Create a new variable `both_tokens` to represent the total number of tokens in the collection. Create a variable `vocabulary_size` whose value is the total number of distinct word types in the collection. Print the vocabulary size.

In [20]:
both_counts = pos_word_counts + neg_word_counts
# print(both_counts)
both_tokens = pos_tokens + neg_tokens
print(both_tokens)   
vocabulary_size = len(both_counts)
print(vocabulary_size)

542123
24182


### Q11: Calculate word probability (version 3.0)

Now create a function `smoothed_word_log_prob` that takes the same arguments as `word_prob` (word, counter, sum of the counter), but adds 1 to the count for the word (don't modify the counter, just add 1 in your function). Previously we divided the word count by the sum of all the counts in the counter, which guaranteed that the sum of the probability of all word types was 1.0. Now that we are adding 1 to each word, what should the denominator be so that this new probability distribution will sum to 1.0?

Use this function to print the log probability of the words "delicious", "manager", "edgy", and "vile" in both sub-collections. (These function calls should have the exact same arguments as the previous blocks.)

In [21]:
def smoothed_word_log_prob(word, counter, sum):
    dict_counter = dict(counter)
    count = 0
    for i in dict_counter:
        if i == word:
            count = dict_counter[i]
            count = count + 1
            newsum = sum - 1
        
    return word_log_prob(word, counter, newsum)
        
print(smoothed_word_log_prob("delicious", pos_word_counts, pos_tokens))
print(smoothed_word_log_prob("manager", pos_word_counts, pos_tokens))

-6.659351646405353
-8.689377828521838


### Q12: Test your classifier!

Now let's put it all together. Given a review, we want to know whether that review is more likely to be positive or negative. To answer this question we will calculate the log of the ratio between $P(positive | words\ in\ review)$ and $P(negative | words\ in\ review)$. This ratio is simpler than the two individual. Why?

answer here:

Calculate this ratio for the review "the ambience was edgy but the food was disgusting". Use the probability that a review is positive or negative that you calculated earlier as the prior probabilities $P(positive)$ and $P(negative)$, but this time use the log of those probabilities. Print the resulting log probability ratio.

* How should you interpret the log probability ratio?
* Which category is more likely?
* Does the prior probability matter?
* Suggest a prior probability that would change our decision, and explain why.

Answers here: